In [91]:
import yfinance as yf
import pandas as pd

stock = yf.Ticker("SPY")
dividends = stock.dividends

In [95]:
spot_prices = stock.history(period="1y")["Close"].to_frame()

In [99]:
spot_prices.shape[0]

252

In [105]:
daily_returns = spot_prices.pct_change().dropna()
daily_returns.head()

,Close
Date,
2023-11-07 00:00:00-05:00,0.002846
2023-11-08 00:00:00-05:00,0.000732
2023-11-09 00:00:00-05:00,-0.007799
2023-11-10 00:00:00-05:00,0.015605
2023-11-13 00:00:00-05:00,-0.000953


In [109]:
historical_volatility = daily_returns.std() * (252**0.5)
historical_volatility

Close    0.120413
dtype: float64

In [111]:
expiration_dates = stock.options
expiration_dates

('2024-11-05',
 '2024-11-06',
 '2024-11-07',
 '2024-11-08',
 '2024-11-11',
 '2024-11-12',
 '2024-11-13',
 '2024-11-14',
 '2024-11-15',
 '2024-11-22',
 '2024-11-29',
 '2024-12-06',
 '2024-12-13',
 '2024-12-20',
 '2024-12-31',
 '2025-01-17',
 '2025-01-31',
 '2025-02-28',
 '2025-03-21',
 '2025-03-31',
 '2025-04-17',
 '2025-04-30',
 '2025-06-20',
 '2025-06-30',
 '2025-08-15',
 '2025-09-19',
 '2025-09-30',
 '2025-12-19',
 '2026-01-16',
 '2026-06-18',
 '2026-12-18',
 '2027-01-15')

In [113]:
calls_all = [stock.option_chain(date).calls for date in expiration_dates]
puts_all = [stock.option_chain(date).puts for date in expiration_dates]

In [121]:
puts_all

[        contractSymbol             lastTradeDate  strike  lastPrice    bid  \
 0   SPY241105P00300000 2024-11-05 16:55:06+00:00   300.0       0.01   0.00   
 1   SPY241105P00340000 2024-10-25 13:30:00+00:00   340.0       0.01   0.00   
 2   SPY241105P00360000 2024-10-24 13:35:00+00:00   360.0       0.02   0.00   
 3   SPY241105P00370000 2024-10-23 18:49:25+00:00   370.0       0.02   0.00   
 4   SPY241105P00380000 2024-10-31 18:41:44+00:00   380.0       0.01   0.00   
 ..                 ...                       ...     ...        ...    ...   
 84  SPY241105P00600000 2024-11-01 13:56:33+00:00   600.0      26.79  21.89   
 85  SPY241105P00601000 2024-10-25 20:05:00+00:00   601.0      22.08  23.30   
 86  SPY241105P00602000 2024-10-25 20:05:00+00:00   602.0      23.09  23.92   
 87  SPY241105P00605000 2024-10-30 19:33:38+00:00   605.0      24.19  26.95   
 88  SPY241105P00610000 2024-11-05 20:15:30+00:00   610.0      34.47  31.94   
 
       ask    change  percentChange  volume  openI

In [123]:
calls_dict = {date: stock.option_chain(date).calls for date in expiration_dates}
puts_dict = {date: stock.option_chain(date).puts for date in expiration_dates}

In [124]:
calls_dict

{'2024-11-05':         contractSymbol             lastTradeDate  strike  lastPrice     bid  \
 0   SPY241105C00300000 2024-11-05 14:30:05+00:00   300.0     270.12  276.80   
 1   SPY241105C00410000 2024-11-04 17:29:45+00:00   410.0     159.99  166.80   
 2   SPY241105C00480000 2024-11-04 19:52:41+00:00   480.0      90.88   96.77   
 3   SPY241105C00496000 2024-11-05 20:47:39+00:00   496.0      78.86   80.80   
 4   SPY241105C00500000 2024-11-05 14:37:37+00:00   500.0      75.26   76.80   
 ..                 ...                       ...     ...        ...     ...   
 74  SPY241105C00640000 2024-10-28 14:22:44+00:00   640.0       0.01    0.00   
 75  SPY241105C00645000 2024-10-30 14:08:14+00:00   645.0       0.01    0.00   
 76  SPY241105C00660000 2024-10-31 15:43:33+00:00   660.0       0.01    0.00   
 77  SPY241105C00665000 2024-11-05 14:40:39+00:00   665.0       0.01    0.00   
 78  SPY241105C00670000 2024-11-04 16:45:43+00:00   670.0       0.01    0.00   
 
        ask    change  p

In [127]:
from datetime import datetime

expiration_df = pd.DataFrame({'expiration': expiration_dates})
expiration_df['time_to_expiry'] = expiration_df['expiration'].apply(
    lambda x: (datetime.strptime(x, '%Y-%m-%d') - datetime.now()).days / 365
)

In [133]:
# Add expiration column to each DataFrame in calls_dict and puts_dict
for date, df in calls_dict.items():
    df['expiration'] = date

for date, df in puts_dict.items():
    df['expiration'] = date

# Concatenate all DataFrames from calls_dict and puts_dict
calls_all = pd.concat(calls_dict.values())
puts_all = pd.concat(puts_dict.values())

# Combine calls and puts into a single DataFrame
options_data = pd.concat([calls_all, puts_all])

# Select desired columns
options_data = options_data[['contractSymbol', 'strike', 'lastPrice', 'impliedVolatility', 'expiration']]


In [153]:
options_data.iloc[0]

contractSymbol       SPY241105C00300000
strike                            300.0
lastPrice                        270.12
impliedVolatility              5.812503
expiration                   2024-11-05
Name: 0, dtype: object